In [ ]:
import numpy as np
from loguru import logger
from glob import glob
import os
import os.path as osp
import mmcv
from avcv.coco import *

import argparse

datasets = [
    ('/data/DMS_Behavior_Detection/Action_Eating_frames/annotations/instances_default.json', 
        '/data/DMS_Behavior_Detection/Action_Eating_frames/images'),
    ('/data/DMS_Behavior_Detection/Mobile_Cigarret_DB/mobile_cigarette_train_081522_finetuning.json', 
        '/data/DMS_Behavior_Detection/Mobile_Cigarret_DB/'),
    ('/data/DMS_Behavior_Detection/Mobile_Cigarret_DB/mobile_cigarette_val_080422.json', 
        '/data/DMS_Behavior_Detection/Mobile_Cigarret_DB/'),
    ('/data/public/coco/annotations/instances_train2017.json', '/data/public/coco/train2017/'),
    ('/data/public/coco/annotations/instances_val2017.json', '/data/public/coco/val2017/')
]
new_root_dir = '/data/DMS_Behavior_Detection/merge-phone-cigaret-food/'

mmcv.mkdir_or_exist(new_root_dir)

cats = []
for ann, _ in datasets:
    for cat in mmcv.load(ann)['categories']:
        cats.append(cat['name'])

print(sorted(set(cats)))

['airplane', 'apple', 'backpack', 'banana', 'baseball bat', 'baseball glove', 'bear', 'bed', 'bench', 'bicycle', 'bird', 'boat', 'book', 'bottle', 'bowl', 'broccoli', 'bus', 'cake', 'cam', 'car', 'carrot', 'cat', 'cell phone', 'chair', 'cigarette', 'clock', 'couch', 'cow', 'cup', 'dining table', 'dog', 'donut', 'elephant', 'face', 'face_visible', 'fire hydrant', 'food/drink', 'fork', 'frisbee', 'giraffe', 'hair drier', 'hand', 'handbag', 'horse', 'hot dog', 'keyboard', 'kite', 'knife', 'laptop', 'left eye close', 'left eye na', 'left eye open', 'lomui', 'longmayphai', 'longmaytrai', 'mask', 'matphai', 'mattrai', 'microwave', 'moingoai', 'moitrong', 'motorcycle', 'mouse', 'mouth', 'mouth close', 'mouth open', 'orange', 'oven', 'parking meter', 'person', 'phone', 'pizza', 'potted plant', 'refrigerator', 'remote', 'right eye close', 'right eye na', 'right eye open', 'sandwich', 'scissors', 'sheep', 'sink', 'skateboard', 'skis', 'snowboard', 'songmui', 'spoon', 'sports ball', 'stop sign', 

In [ ]:
category_old_to_new = {
    'cigarette':'cigarette',
    'food/drink':'food/drink',
    'phone':'phone',
    'cell phone': 'phone',
    'donut':'food/drink',
    'hot dog':'food/drink',
    'pizza':'food/drink',
    'wine glass':'food/drink',
    'orange':'food/drink',
    'cake':'food/drink',
    'banana':'food/drink',
    'sandwich':'food/drink',
    'apple':'food/drink',
}
categories = []
used_names = set()
for id, name in enumerate(category_old_to_new.values()):
    if not name in used_names:
        categories += [{'id':id, 'name':name} ]
        used_names.add(name)
    
name2cat = {cat['name']:cat for cat in categories}

In [ ]:
categories

[{'id': 0, 'name': 'cigarette'},
 {'id': 1, 'name': 'food/drink'},
 {'id': 2, 'name': 'phone'}]

In [ ]:

from avcv.all import *


out_dict = dict(
    images=[],
    annotations=[],
    categories=categories

)

new_img_dir = osp.join(new_root_dir, 'images')
for json, img_dir in datasets:
    cc = CocoDataset(json, img_dir)
    # cc.visualize();
    for img_id in cc.img_ids:
        img = cc.gt.imgs[img_id].copy()
        new_img_id = len(out_dict['images'])
        new_img = img.copy()
        new_img['id'] = new_img_id
        file_name = os.path.abspath(osp.join(cc.img_dir, img['file_name']))
        rel_path = os.path.relpath(file_name, new_img_dir)
        new_img['file_name'] = rel_path
        

        take = False
        for ann in cc.gt.imgToAnns[img_id]:
            ann = ann.copy()
            ann['image_id'] = new_img_id
            old_cat = cc.gt.cats[ann['category_id']]
            if old_cat['name'] in category_old_to_new:
                new_cat_name = category_old_to_new[old_cat['name']]
                new_cat = name2cat[new_cat_name]
                ann['category_id'] = new_cat['id']
                ann['id'] = len(out_dict['annotations'])
                out_dict['annotations'].append(ann)
                take = True
        if not '/public/coco' in new_img['file_name'] or take:
            out_dict['images'].append(new_img)
        # if get_name(file_name) == 'vietanh_phoneusage_0012':
        #     vis = cc.visualize(img_id)
        #     new_cc = CocoDataset(out_dict, new_img_dir)
        #     new_vis = new_cc.visualize(new_img_id)

        #     mmcv.imwrite(np.concatenate([vis, new_vis], 1), 'vis.jpg')
            # import ipdb; ipdb.set_trace()


train_coco, val_coco = split_coco(out_dict)



# cmd = f"python tools/visualize_coco.py {train_json_out} --imgs {new_img_dir}"
# logger.info(cmd)


2022-09-16 07:15:31.498 | INFO     | avcv.coco:split_coco:470 - Spliting ratio 0.85 -> 30615 training and 5403 testing


In [ ]:
len(train_coco['images']), len(val_coco['images'])

(30615, 5403)

In [ ]:
train_json_out = osp.join(new_root_dir, 'annotations/train_with_coco.json')
val_json_out = osp.join(new_root_dir, 'annotations/val_with_coco.json')
mmcv.dump(train_coco, train_json_out)
mmcv.dump(val_coco, val_json_out)